# SHM Extension Test Notebook

This notebook tests the SHM Function Selector extension.

In [7]:
# Test SHM function discovery
import shmtools
print('SHMTools version:', getattr(shmtools, '__version__', 'unknown'))

# List available modules
modules = ['core', 'features', 'classification']
for mod in modules:
    if hasattr(shmtools, mod):
        print(f'✓ {mod} module available')
    else:
        print(f'- {mod} module not available')

SHMTools version: 0.1.0
✓ core module available
✓ features module available
✓ classification module available


In [11]:
%%javascript
console.log('Starting SHM Extension Test...');

// Create mock Jupyter environment if needed
if (typeof Jupyter === 'undefined') {
    window.Jupyter = {
        notebook: {
            base_url: '',
            get_selected_cell: function() {
                return {
                    cell_type: 'code',
                    get_text: function() { return ''; },
                    set_text: function(text) { 
                        console.log('Cell text set to:', text); 
                    }
                };
            },
            insert_cell_below: function(type) {
                console.log('Inserting cell of type:', type);
                return {
                    set_text: function(text) {
                        console.log('New cell content:', text);
                        // Create a visible result
                        var pre = document.createElement('pre');
                        pre.style.background = '#f8f9fa';
                        pre.style.padding = '10px';
                        pre.style.border = '1px solid #ddd';
                        pre.style.marginTop = '10px';
                        pre.textContent = text;
                        document.body.appendChild(pre);
                    }
                };
            }
        }
    };
}

// Mock SHM functions data
var mockFunctions = [
    {
        name: 'psd_welch',
        category: 'Core - Spectral Analysis',
        display_name: 'Power Spectral Density (Welch)',
        description: 'Estimate power spectral density using Welch method',
        parameters: [
            {name: 'x', type: 'numpy.ndarray', optional: false},
            {name: 'fs', type: 'float', default: '1.0', optional: true},
            {name: 'nperseg', type: 'int', default: 'None', optional: true}
        ]
    },
    {
        name: 'ar_model',
        category: 'Features - Time Series Models',
        display_name: 'AR Model Parameters & Residuals', 
        description: 'Estimate autoregressive model parameters',
        parameters: [
            {name: 'X', type: 'numpy.ndarray', optional: false},
            {name: 'ar_order', type: 'int', default: '5', optional: true}
        ]
    },
    {
        name: 'learn_pca',
        category: 'Classification - Outlier Detection',
        display_name: 'Learn PCA Model',
        description: 'Learn PCA model for outlier detection',
        parameters: [
            {name: 'X', type: 'numpy.ndarray', optional: false},
            {name: 'per_var', type: 'float', default: '0.9', optional: true}
        ]
    },
    {
        name: 'bandpass_filter',
        category: 'Core - Filtering',
        display_name: 'Bandpass Filter',
        description: 'Apply bandpass filter to signal',
        parameters: [
            {name: 'x', type: 'numpy.ndarray', optional: false},
            {name: 'lowcut', type: 'float', optional: false},
            {name: 'highcut', type: 'float', optional: false},
            {name: 'fs', type: 'float', optional: false}
        ]
    }
];

// Function to generate code
function generateCode(func) {
    var lines = ['# ' + func.description];
    var params = [];
    
    func.parameters.forEach(function(param) {
        var paramStr = param.name + '=';
        if (param.default && param.default !== 'None') {
            paramStr += param.default;
        } else {
            paramStr += 'None  # ' + (param.optional ? 'TODO' : 'REQUIRED') + ': ' + param.type;
        }
        params.push(paramStr);
    });
    
    var outputs = 'result';
    if (func.name.includes('ar_model')) {
        outputs = 'features, residuals';
    } else if (func.name.includes('psd')) {
        outputs = 'frequencies, power_spectrum';
    } else if (func.name.includes('learn')) {
        outputs = 'model';
    } else if (func.name.includes('filter')) {
        outputs = 'filtered_signal';
    }
    
    var code = outputs + ' = shmtools.' + func.name + '(\n    ' + 
               params.join(',\n    ') + '\n)';
    lines.push(code);
    
    return lines.join('\n');
}

// Create UI elements
console.log('Creating SHM Functions dropdown...');

// Create dropdown button
var dropdown = document.createElement('div');
dropdown.className = 'dropdown';
dropdown.style.margin = '10px';
dropdown.innerHTML = `
    <button class='btn btn-primary dropdown-toggle' type='button' data-bs-toggle='dropdown' style='font-size: 14px;'>
        🔧 SHM Functions
    </button>
    <ul class='dropdown-menu' id='shm-functions-list' style='max-height: 400px; overflow-y: auto; min-width: 300px;'>
        <li><h6 class='dropdown-header'>📊 Available SHM Functions</h6></li>
    </ul>
`;

// Add to page
document.body.appendChild(dropdown);

// Group functions by category
var categories = {};
mockFunctions.forEach(function(func) {
    if (!categories[func.category]) {
        categories[func.category] = [];
    }
    categories[func.category].push(func);
});

// Populate dropdown
var list = document.getElementById('shm-functions-list');
Object.keys(categories).forEach(function(category) {
    // Add category header
    var header = document.createElement('li');
    header.innerHTML = '<h6 class="dropdown-header" style="color: #337ab7; font-weight: bold;">' + category + '</h6>';
    list.appendChild(header);
    
    // Add functions in this category
    categories[category].forEach(function(func) {
        var item = document.createElement('li');
        var link = document.createElement('a');
        link.className = 'dropdown-item';
        link.href = '#';
        link.textContent = func.display_name;
        link.title = func.description;
        link.onclick = function(e) {
            e.preventDefault();
            var code = generateCode(func);
            console.log('Generated code for ' + func.name + ':', code);
            
            // Use Jupyter to insert code
            if (Jupyter && Jupyter.notebook) {
                var cell = Jupyter.notebook.insert_cell_below('code');
                cell.set_text(code);
            }
            
            alert('✅ Code generated for: ' + func.display_name + '\n\nCheck the new cell below!');
        };
        item.appendChild(link);
        list.appendChild(item);
    });
    
    // Add separator
    var divider = document.createElement('li');
    divider.innerHTML = '<hr class="dropdown-divider">';
    list.appendChild(divider);
});

// Add Bootstrap dropdown functionality manually if needed
dropdown.querySelector('button').onclick = function() {
    var menu = dropdown.querySelector('.dropdown-menu');
    menu.style.display = menu.style.display === 'block' ? 'none' : 'block';
};

// Add styling
var style = document.createElement('style');
style.textContent = `
    .dropdown-menu {
        border: 1px solid #ddd;
        border-radius: 4px;
        box-shadow: 0 2px 10px rgba(0,0,0,0.2);
        background: white;
    }
    .dropdown-item:hover {
        background-color: #f8f9fa !important;
        color: #337ab7;
    }
    .btn-primary {
        background-color: #337ab7;
        border-color: #337ab7;
    }
`;
document.head.appendChild(style);

console.log('✅ SHM Extension test complete!');
console.log('📋 Functions loaded: ' + mockFunctions.length);
console.log('📁 Categories: ' + Object.keys(categories).length);
console.log('👆 Click the "🔧 SHM Functions" button above to test!');

<IPython.core.display.Javascript object>

## Manual Testing

After running the cells above:

1. **Check toolbar** - Look for 'SHM Functions' dropdown button
2. **Try dropdown** - Click to see function categories
3. **Test insertion** - Click a function to insert code

If extension doesn't appear, check browser console (F12) for error messages.